In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp


In [3]:
sns.set_context('poster')

### Catalytic Promiscuity in the Biosynthesis of Cyclic Peptide in Planktonic Marine Cyanobacteria, PNAS2010
Detection of procM and procA gene sequences in marine cyanobacterial genomes.
1. **procM** gene sequences were detected in marine cyanobacterial genomes using the **pre-computed phylogenetic tree viewer in the microbesonline database** (http://www.microbesonline.org (3), which is computed using FastBLAST (4), see http://www.microbesonline.org/treebrowseHelp.html for additional details). 
2. **procA** sequences were detected in the genomes of Prochlorococcus **MIT9313** by **manually searching for short ORFs close to the procM gene** that encoded a **peptide** with a **leader sequence** followed by a **double-glycine** protease recognition site and a **short** core peptide rich in **Cys, Ser and Thr** residues (Fig. 1C). 
3. Once several such putative lantipeptides were discovered, we used **BLASTp** in both **microbesonline** and the **NCBI nr** database (with a cutoff **e-value of 10**) to detect other, similar genes in cyanobacterial genomes
4.  **manually** checked the resulting peptide sequences to verify that they agreed with the definition of procAs, as described above. 
5. Figure 1 and Figure S1 were produced using IMG (http://img.jgi.doe.gov/cgi-bin/pub/main.cgi).

### Evolutionary radiation of lanthipeptides in marine cyanobacteria, PNAPLUS 2017
1. Annotation of **prochlorosin** biosynthesis genes was performed manually using **BLAST** (12). 
2. The sequences of the **ProcM, LanT and LanOM** proteins from Prochlorococcus **MIT9313** were used as queries for the search of homologous proteins. 
3. Top hits were further inspected against the **NCBI Conserved Domain Database (CDD)** (13) to verify the presence of signature domains.
4. The **tBLASTn** algorithm was used to search **procA and procA pseudogenes** using a **consensus sequence** derived from **29 prochlorosin leader peptide** sequences from Prochlorococcus **MIT9313**. 
5. BLAST hits with **>45% identity** and **E-val <0.001** were inspected against the CDD to confirm the presence of the **nif11-like leader peptide domain**.


![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
def find_split_index(aa_sequence):
    gg_index = aa_sequence.find('GG')
    return gg_index
    ga_index = aa_sequence.find('GA')
    split_index = -1
    if gg_index == -1:
        split_index = ga_index
    elif ga_index == -1:
        split_index = gg_index
    else:
        split_index = min(gg_index, ga_index)
    return split_index
    
def analyze_procm_data(row):
    aa_sequence = str(row['aa_sequence'])
    function = str(row['function'])
    aa_length = len(aa_sequence)
    split_index = find_split_index(aa_sequence)
    core_sequence = None
    core_length = None
    count_dict = dict()
    precent_dict = dict()
    is_lanm = False
    if function == 'Lanthionine biosynthesis protein LanM':
        is_lanm = True

    if split_index != -1:
        core_sequence = aa_sequence[split_index+2:]
        core_length = len(core_sequence)
        if core_length > 0:
            count = Counter(core_sequence)
            count_dict = {f'core_count_{k}' : v for k,v in count.items()}
            precent_dict = {f'core_precent_{k}' : v/core_length for k,v in count.items()}
    return row.append(pd.Series({
        'aa_length' : aa_length,
        'split_index' : split_index,
        'core_sequence' : core_sequence,
        'core_length' : core_length,
        'is_lanm' : is_lanm,
        **count_dict, 
        **precent_dict, 
    }))

In [5]:
import subprocess
def run_diamond_to_create_db():
    dbname='pro_faa'
    cwd = os.getcwd()
    in_fasta_dpath = os.path.join(cwd, 'data', 'all_Prochlorococcus_ORFs_faa')
    
    diamond_exe = os.path.join(cwd, 'bin','diamond.exe')
    out_dpath = os.path.join(cwd,'diamond_tmp')
    diamond_params = [#'TYPE', os.path.join(in_fasta_dpath, '*'), 
                      #'|', 
                      diamond_exe, 'makedb', 
                      '--in', os.path.join(in_fasta_dpath, 'all_faa.faa'), 
                      '-d', os.path.join(out_dpath, dbname)
    ]
    try:
        os.makedirs(out_dpath, exist_ok=True)
        retcode = subprocess.check_call(' '.join(diamond_params), shell=True)
        if retcode < 0:
            print("Child was terminated by signal", -retcode)
    except OSError as e:
        print("Execution failed:", e)
    except subprocess.CalledProcessError as e:
        print("Execution failed:", e)
    


In [6]:
import subprocess
def _write_fasta(fname, sequence):
    with open(fname, 'w') as f:
        f.write(f'> tmp\n{sequence}\n')
        
def _read_diamond_results(fname):
    diamond_columns = [ 'gene_id', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart',
                   'send', 'evalue', 'bitscore']
    dout_df = pd.read_csv(fname, sep='\t', 
                     names=diamond_columns)
    return dout_df.reset_index(drop=True)

def run_diamond_to_align_sequence(aa_sequence):
    dbname='pro_faa'
    cwd = os.getcwd()
    in_fasta_dpath = os.path.join(cwd, 'data', 'all_Prochlorococcus_ORFs_faa')
    
    diamond_exe = os.path.join(cwd, 'bin','diamond.exe')
    out_dpath = os.path.join(cwd,'diamond_tmp')
    out_fpath = os.path.join(out_dpath, 'diamond_out.txt')
    tmp_fpath = os.path.join(out_dpath, 'tmp.faa')

    os.makedirs(out_dpath, exist_ok=True)
    _write_fasta(tmp_fpath, aa_sequence)
    diamond_params = [#'TYPE', os.path.join(in_fasta_dpath, '*'), 
                      #'|', 
                      diamond_exe, 'blastp', 
                      '-q', tmp_fpath, 
                      '-d', os.path.join(out_dpath, dbname),
                      '-o', out_fpath,
    ]
    try:
        subprocess.check_call(' '.join(diamond_params), shell=True)
        return _read_diamond_results(out_fpath)
    except OSError as e:
        print("Execution failed:", e)
    except subprocess.CalledProcessError as e:
        print("Execution failed:", e)
    


In [7]:
genomes_dpath = r'data\detailed_Prochlorococcus_genome_annotations'
genome_fnames = [ n for n in os.listdir(genomes_dpath) if n.endswith('.txt')]
def _load(fname):
    df = pd.read_csv(os.path.join(genomes_dpath, fname), sep='\t')
    df['genome'] = os.path.basename(os.path.splitext(fname)[0])
    return df
genome_df = pd.concat([_load(fname) for fname in genome_fnames])


In [8]:
genome_df  = genome_df.reset_index(drop=True)

In [9]:
genome_df.head()

,contig_id,gene_id,feature_id,type,location,start,stop,strand,function,aliases,figfam,evidence_codes,nucleotide_sequence,aa_sequence,genome
0,AS9601,PAS9601_0001,fig|1218.83.peg.1,peg,AS9601_168_1325,168,1325,+,DNA polymerase III beta subunit (EC 2.7.7.7),NaN,FIG00066425,isu;DNA-replication isu;DNA_replication_cluster_1,atggaaattatttgtaatcaaaatgaattaaataatgctatacaac...,MEIICNQNELNNAIQLVSKAVASRPTHPILANILLTADEGTNKISV...,AS9601
1,AS9601,PAS9601_0002,fig|1218.83.peg.2,peg,AS9601_1327_2034,1327,2034,+,RNA metabolism-related protein,NaN,NaN,NaN,ttgaaattacctaaagaaattttattaagtgaattattaaattata...,MKLPKEILLSELLNYIVKGNMVLNYGNGENVWMHPPVHRILGWYSR...,AS9601
2,AS9601,PAS9601_0003,fig|1218.83.peg.3,peg,AS9601_2038_4377,2038,4377,+,"Phosphoribosylformylglycinamidine synthase, sy...",NaN,FIG01303876,icw(1);De_Novo_Purine_Biosynthesis,atgataaatcatgaaaataatgatctatttgatcttaatgaagcat...,MINHENNDLFDLNEALKVENLTLNDYEEICKRLKRKPNRTELGMFG...,AS9601
3,AS9601,PAS9601_0004,fig|1218.83.peg.4,peg,AS9601_4425_5885,4425,5885,+,Amidophosphoribosyltransferase (EC 2.4.2.14),NaN,FIG00000179,isu;YgfZ isu;De_Novo_Purine_Biosynthesis,atgtgcggaatagttggaatcgtttcttcgaatgatgtaaatcaac...,MCGIVGIVSSNDVNQQIYDSLLLLQHRGQDSTGIATMENTVFHIHK...,AS9601
4,AS9601,PAS9601_0005,fig|1218.83.peg.5,peg,AS9601_8323_5882,8323,5882,-,DNA gyrase subunit A (EC 5.99.1.3),NaN,FIG00000080,idu(1);DNA_gyrase_subunits idu(1);DNA_topoisom...,atggataagaaaaatttcacttccatatcacttcaagaagaaatgc...,MDKKNFTSISLQEEMQRSYLEYAMSVIVGRALPDARDGLKPVQRRI...,AS9601


In [47]:
genome_df.iloc[59275].aa_sequence

'MSEEQLKAFIAKVQADTSLQEQLKAEGADVVAIAKAAGFSITTEDLEKEHRQTLSDDDLEGVAGGFFCVQGTANRFTINVC'

In [10]:
df = genome_df.apply(analyze_procm_data, axis=1)

In [22]:
def is_candidate(r, 
                 max_aa_sequence_length = 600,
                 min_core_length = 5, 
                 max_core_length = 200, 
                 min_split_index = 5,
                 min_prec_CST = 0.1
                ):
    row = r.fillna(0)
    if ((row.split_index > min_split_index) and 
        (row.aa_length < max_aa_sequence_length) and 
        (row.core_length < max_core_length) and
        (row.core_length > min_core_length) and
        (row.core_precent_C + row.core_precent_T + row.core_precent_S > min_prec_CST)
       ):
        return True
    return False
        

In [23]:
df['is_candidate'] = df.apply(is_candidate, axis=1)

In [28]:
df.groupby('genome').is_candidate.sum()

genome
AS9601     384.0
EQPAC1     391.0
GP2        368.0
HNLC1      353.0
HNLC2      347.0
LG         429.0
MED4       392.0
MIT0601    432.0
MIT0602    433.0
MIT0603    431.0
MIT0604    417.0
MIT0701    636.0
MIT0702    623.0
MIT0703    624.0
MIT0801    436.0
MIT9107    386.0
MIT9116    387.0
MIT9123    386.0
MIT9201    389.0
MIT9202    381.0
MIT9211    419.0
MIT9215    392.0
MIT9301    394.0
MIT9302    408.0
MIT9303    639.0
MIT9311    389.0
MIT9312    392.0
MIT9313    637.0
MIT9314    393.0
MIT9321    386.0
MIT9322    394.0
MIT9401    389.0
MIT9515    374.0
NATL1A     437.0
NATL2A     406.0
PAC1       439.0
SB         381.0
SS120      423.0
SS2        429.0
SS35       427.0
SS51       421.0
SS52       429.0
UH18301    392.0
Name: is_candidate, dtype: float64

In [31]:
selected = df.loc[df.function.str.contains('lanm', case=False)]

In [32]:
selected

,aa_length,aa_sequence,aliases,contig_id,core_count_A,core_count_C,core_count_D,core_count_E,core_count_F,core_count_G,...,is_lanm,location,nucleotide_sequence,split_index,start,stop,strand,type,dist_after_cumsum,is_candidate
23489,1082,MGAAEPLALQSSWKSLWLEAVAADEPHKFKRRLEWDGLTESEFHAA...,NaN,contig00017,91.0,19.0,50.0,45.0,27.0,67.0,...,True,contig00017_4336_7584,atgggagctgctgagcctttggcacttcaatcatcttggaaatccc...,255,4336,7584,+,peg,True,False
26640,1082,MGAAEPLALQSSWKSLWLEAVAADEPHKFKRRLEWDGLTESEFHAA...,NaN,contig00012,91.0,19.0,50.0,45.0,27.0,67.0,...,True,contig00012_65496_62248,atgggagctgctgagcctttggcacttcaatcatcttggaaatccc...,255,65496,62248,-,peg,True,False
30660,1082,MGAAEPLALQSSWKSLWLEAVAADEPHKFKRRLEWDGLTESEFHAA...,NaN,contig00032,91.0,19.0,50.0,45.0,27.0,67.0,...,True,contig00032_21482_18234,atgggagctgctgagcctttggcacttcaatcatcttggaaatccc...,255,21482,18234,-,peg,True,False
53940,1074,MGAAEPLTSPTSWKTSWLAAIAPDEPHKFDRRLEWDELSEENFFAA...,NaN,MIT9303,100.0,15.0,47.0,43.0,29.0,66.0,...,True,MIT9303_1865514_1868738,atgggagctgctgagcctttgacaagtccaacatcttggaaaacca...,249,1865514,1868738,+,peg,True,False
59279,1074,MGAAEPLESPSSWKTSWLAAIAPDEPHKFDRRLEWDELSEENFFAA...,NaN,MIT9313,93.0,15.0,47.0,45.0,29.0,68.0,...,True,MIT9313_283359_280135,atgggagctgccgagcctttggaaagtccatcatcttggaaaacca...,249,283359,280135,-,peg,True,False


In [33]:
selected = df.loc[df.gene_id == 'PMIT9313_0302']

In [44]:
selected.gene_id


59279    PMIT9313_0302
Name: gene_id, dtype: object

In [43]:
selection_win = 20
idx = selected.index[0]
idx
vib = df.iloc[idx - selection_win: idx + selection_win + 1 ]
vib[vib.is_candidate][['gene_id', 'function']]

,gene_id,function
59261,PMIT9313_0284,hypothetical protein
59262,PMIT9313_0285,possible Indole-3-glycerol phosphate synthase
59264,PMIT9313_0287,possible Indole-3-glycerol phosphate synthase
59267,PMIT9313_0290,hypothetical protein
59268,PMIT9313_0291,possible [2Fe-2S] binding domain
59269,PMIT9313_0292,hypothetical protein
59271,PMIT9313_0294,possible (M20568) ORF 11 [Azotobacter vinelandii]
59273,PMIT9313_0296,hypothetical protein
59274,PMIT9313_0297,hypothetical protein
59275,PMIT9313_0298,possible (M20568) ORF 11 [Azotobacter vinelandii]


277780 .. 278052 (-) 
276160 .. 276447 (-) 
275220 .. 275483 (-) 
274751 .. 274996 (-) 
274290 .. 274553 (-) 
273356 .. 273631 (-) 
272419 .. 272715 (-) 

In [20]:
row = row.fillna(0)

In [40]:
df.columns

Index(['aa_length', 'aa_sequence', 'aliases', 'contig_id', 'core_count_A',
       'core_count_C', 'core_count_D', 'core_count_E', 'core_count_F',
       'core_count_G', 'core_count_H', 'core_count_I', 'core_count_K',
       'core_count_L', 'core_count_M', 'core_count_N', 'core_count_P',
       'core_count_Q', 'core_count_R', 'core_count_S', 'core_count_T',
       'core_count_V', 'core_count_W', 'core_count_X', 'core_count_Y',
       'core_count_x', 'core_length', 'core_precent_A', 'core_precent_C',
       'core_precent_D', 'core_precent_E', 'core_precent_F', 'core_precent_G',
       'core_precent_H', 'core_precent_I', 'core_precent_K', 'core_precent_L',
       'core_precent_M', 'core_precent_N', 'core_precent_P', 'core_precent_Q',
       'core_precent_R', 'core_precent_S', 'core_precent_T', 'core_precent_V',
       'core_precent_W', 'core_precent_X', 'core_precent_Y', 'core_precent_x',
       'core_sequence', 'evidence_codes', 'feature_id', 'figfam', 'function',
       'gene_id', 'geno

In [67]:
run_diamond_to_create_db('pro_faa')

In [76]:
run_diamond_to_align_sequence('MSEEQLKAFIAKVQADTSLQEQLKAEGADVVAIAKAAGFSITTEDLEKEHRQTLSDDDLEGVAGGFFCVQGTANRFTINVC')


In [80]:
diamond_columns = [ 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart',
                   'send', 'evalue', 'bitscore']
dout_df = pd.read_csv(r'C:\Users\wosnat\Documents\GitHub\ripp\diamond_tmp\diamond_out.txt', sep='\t', 
                     names=diamond_columns)

In [83]:
dout_df.reset_index(drop=True).head()

,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,PMIT9313_0298,100.0,81,0,0,1,81,1,81,3.100000e-39,158.7
1,PMIT9313_2802,92.6,68,4,1,1,68,1,67,6.800000e-26,114.4
2,PMIT9313_1204,95.4,65,2,1,1,65,1,64,2.600000e-25,112.5
3,PMIT9313_2803,87.0,69,8,1,1,69,1,68,7.500000e-25,110.9
4,PMIT9313_0287,89.4,66,6,1,1,65,1,66,1.700000e-24,109.8


In [84]:
df.loc[df.gene_id == 'PMIT9313_0298'].T

,59275
aa_length,81
aa_sequence,MSEEQLKAFIAKVQADTSLQEQLKAEGADVVAIAKAAGFSITTEDL...
aliases,NaN
contig_id,MIT9313
core_count_A,6
core_count_C,2
core_count_D,5
core_count_E,4
core_count_F,4
core_count_G,5


In [86]:
dout_df.shape

(25, 11)

In [87]:
dout_df.sseqid

NaN    PMIT9313_0298
NaN    PMIT9313_2802
NaN    PMIT9313_1204
NaN    PMIT9313_2803
NaN    PMIT9313_0287
NaN    PMIT9313_2818
NaN    PMIT9313_2800
NaN    PMIT9313_2801
NaN    PMIT9313_1203
NaN    PMIT9313_1047
NaN    PMIT9303_0548
NaN    PMIT9303_3044
NaN    PMIT9313_1034
NaN    PMIT9313_1037
NaN    PMIT9313_2798
NaN    PMIT9313_0291
NaN    PMIT9313_2821
NaN    PMIT9303_2965
NaN    PMIT9303_1503
NaN    PMIT9313_1222
NaN    PMIT9313_0290
NaN    PMIT9313_2816
NaN    PMIT9313_0292
NaN    PMIT9313_0294
NaN    PMIT9303_0438
Name: sseqid, dtype: object